# py2cytoscape 0.6.0 New Features

By Keiichiro Ono (kono ucsd edu)

Updated on 7/14/2016

## New in 0.6.0
* Session Saving and loading API
* Bug fixes
* Preset layout utility method

In [ ]:
!python --version

In [ ]:
# Boilerplate: Import required packages
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.util_network import NetworkUtil as util
import py2cytoscape.cytoscapejs as renderer
from IPython.display import Image
import networkx as nx
from networkx.drawing import nx_pydot as pyd
import igraph as ig
import numpy as np

from py2cytoscape.data.style import StyleUtil


## Create client and clean up workspace

In [ ]:
# Create Client
cy = CyRestClient()

# Clear current session
cy.session.delete()

## New: Create network from NumPy *ndarray*

Currently supports **undirected** graphs.

## Supported data types

### 1. Binary matrix (default)
If thearay contains only integers 0 or 1, it will be handles as an ***undirected, unweighted graph***.

#### How to create graph from ndarray
Only upper half of the array will be used for graph generation.  For example this binary array:

```
0, 1, 1, 0
0, 0, 1, 0
0, 0, 0, 1
0, 0, 0, 0
```

will create the graph like this:

In [ ]:
matrix1 = np.array([
    [0, 1, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 0]])

net1 = cy.network.create_from_ndarray(matrix1, name='binary sample')
cy.layout.apply(network=net1)
cy.layout.fit(network=net1)

In [ ]:
Image(net1.get_png(height=400))

### 2. Weights
You can also create weighted egdes.  If the matrix contains floating point numbers, it will be handles as an ***undirected, weighted graph***.  In this case, ***numpy.nan*** represents **edge does not exists**.  Again, only upper half of matrix will be used.  

In [ ]:
matrix_size = 10
weighted = np.empty((matrix_size,matrix_size,))
weighted[:] = np.NAN

row_idx = 0
for row in weighted:
    idx = row_idx
    while idx < matrix_size:
        if idx != row_idx and np.random.randint(2):
            weighted[row_idx][idx] = np.random.randn()
        idx +=1
    row_idx += 1

weight_min = weighted.min() 
weight_max = weighted.max() 

    
net2 = cy.network.create_from_ndarray(weighted, name='weighted sample', weighted=True)
cy.layout.apply(network=net2, name="kamada-kawai")
cy.layout.fit(network=net2)
style_s3 = cy.style.create('Sample3')

defaults = {    
    'NODE_SIZE': 10,
    'NODE_BORDER_WIDTH': 0,
    'NODE_LABEL_FONT_SIZE': 24,
}
style_s3.update_defaults(defaults)

score_to_width = StyleUtil.create_slope(min=weight_min, max=weight_max, values=(1, 5))
style_s3.create_continuous_mapping(column='weight', vp='EDGE_WIDTH', col_type='Double', points=score_to_width)
cy.style.apply(style=style_s3, network=net2)
print(weighted)


In [ ]:
Image(net2.get_png(height=400))

In [ ]:
d1 = np.random.randint(2, size=(10, 10))
d2 = np.ones((10,10))
d3 = np.random.randn(100, 100)
# for x in np.nditer(d3, op_flags=['readwrite']):
#     if np.random.randint(2):
#         x[...] = np.nan
print(d3)

In [ ]:
cy.network.create_from_ndarray(d1, name="Random")
cy.network.create_from_ndarray(d2, name="Complete")
cy.network.create_from_ndarray(d3, name="Weighted", weighted=True)

## Creating networks with NetworkX and igraph

In [ ]:
# Create with NetworkX (Scale-Free graph w/100 nodes)
g = nx.powerlaw_cluster_graph(50,4, 0.5)

# Calculate some graph statistics
deg = nx.degree(g)
btw = nx.betweenness_centrality(g)
nx.set_node_attributes(g, 'degree', deg)
nx.set_node_attributes(g, 'betweenness', btw)

# Create with igraph (Tree w/500 nodes)
g2 = ig.Graph.Tree(400, 5)

## Use 3rd party layout algorithms which does not exists in Cytoscape

In [ ]:
# Graphviz Layout
n_layout = pyd.graphviz_layout(g, prog='dot')

# Reingold-Tilford tree layout with a polar coordinate post-transformation
i_layout = g2.layout("rt_circular")
i_layout.scale(200)

In [ ]:
g_cy = cy.network.create_from_networkx(g)
nodes = g_cy.get_nodes()
idmap = util.name2suid(g_cy)
locations =[]

for k in n_layout.keys():
    v = n_layout[k]
    locations.append([int(idmap[k]), v[0] , v[1]])

cy.layout.apply_from_presets(g_cy, positions=locations)
cy.layout.bundle_edge(g_cy)
my_style = cy.style.create('Curved')
cy.style.apply(style=my_style, network=g_cy)

In [ ]:
Image(g_cy.get_png(height=2000))

In [ ]:
style_for_widget = cy.style.get(my_style.get_name(), data_format='cytoscapejs')
renderer.render(g_cy.get_first_view(), style=style_for_widget['style'], background='radial-gradient(#FFD39B 30%, #FFFFFF 105%)')

In [ ]:
g2_cy = cy.network.create_from_igraph(g2)
nodes = g2_cy.get_nodes()
idmap = util.name2suid(g2_cy)
locations =[]
for i, l in enumerate(i_layout):
    locations.append([int(idmap[i]), l[0] , l[1] ])

cy.layout.apply_from_presets(g2_cy, positions=locations)
style_3 = cy.style.create('Sample3')
cy.style.apply(style=style_3, network=g2_cy)
cy.layout.bundle_edge(g2_cy)

In [ ]:
Image(g2_cy.get_png(height=2000))

In [ ]:

renderer.render(g2_cy.get_first_view(), background='radial-gradient(#FFFFFF 10%, #999999 105%)')

In [ ]:
# Save the session
cy.session.save(file_name='/Users/kono/test34.cys')

In [ ]:
cy.session.open(file_name='/Users/kono/test34.cys')